In [84]:
import pandas as pd
import numpy as np

In [85]:
df = pd.read_csv('/home/antonius/Projects/DS_Projects/Check-ups/3_Machine_learning/data/uk-used-cars/bmw.csv')

In [86]:
df

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
0,5 Series,2014,11200,Automatic,67068,Diesel,125,57.6,2.0
1,6 Series,2018,27000,Automatic,14827,Petrol,145,42.8,2.0
2,5 Series,2016,16000,Automatic,62794,Diesel,160,51.4,3.0
3,1 Series,2017,12750,Automatic,26676,Diesel,145,72.4,1.5
4,7 Series,2014,14500,Automatic,39554,Diesel,160,50.4,3.0
...,...,...,...,...,...,...,...,...,...
10776,X3,2016,19000,Automatic,40818,Diesel,150,54.3,2.0
10777,5 Series,2016,14600,Automatic,42947,Diesel,125,60.1,2.0
10778,3 Series,2017,13100,Manual,25468,Petrol,200,42.8,2.0
10779,1 Series,2014,9930,Automatic,45000,Diesel,30,64.2,2.0


In [87]:
len(df)

10781

### train and test

In [88]:
from sklearn.model_selection import train_test_split

In [89]:
train, test = train_test_split(df, train_size=0.6, random_state=42)

In [90]:
len(train)

6468

In [91]:
len(test)

4313

## Validation for catboost

In [92]:
len(train) / len(df)

0.5999443465355718

In [93]:
len(test) / len (df)

0.4000556534644282

In [94]:
val, test = train_test_split(test, train_size=0.5, random_state=42)

In [95]:
len(train) / len(df)

0.5999443465355718

In [96]:
len(val) / len(df)

0.19998144884519062

In [97]:
len(test) / len(df)

0.20007420461923756

## list of features

In [98]:
train.columns

Index(['model', 'year', 'price', 'transmission', 'mileage', 'fuelType', 'tax',
       'mpg', 'engineSize'],
      dtype='object')

In [99]:
train

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
5133,X6,2015,25000,Semi-Auto,26100,Diesel,205,45.6,3.0
2121,X2,2019,31980,Automatic,4804,Petrol,145,34.0,2.0
6576,X2,2018,22995,Semi-Auto,24136,Petrol,145,38.7,2.0
8330,3 Series,2016,12999,Manual,52224,Diesel,125,62.8,2.0
3161,X3,2019,32980,Semi-Auto,1961,Diesel,150,54.3,2.0
...,...,...,...,...,...,...,...,...,...
5734,3 Series,2018,17995,Semi-Auto,40456,Diesel,145,64.2,2.0
5191,1 Series,2016,19950,Semi-Auto,23210,Petrol,200,39.8,3.0
5390,X2,2019,25450,Semi-Auto,5652,Diesel,145,47.9,2.0
860,X1,2016,19850,Semi-Auto,14542,Diesel,125,60.1,2.0


# Catboost

In [100]:
from catboost import CatBoostRegressor

In [101]:
X = ['year', 'transmission', 'engineSize']

cat_features = ['transmission']

y = ['price']

In [102]:
parameters = {'cat_features': cat_features,
              'eval_metric': 'MAPE',
              'random_seed':42,
              'verbose':100}

In [103]:
model = CatBoostRegressor(**parameters)

In [104]:
model.fit(train[X],train[y],eval_set=(val[X],val[y]))

Learning rate set to 0.068263
0:	learn: 0.4603375	test: 0.4555509	best: 0.4555509 (0)	total: 3.49ms	remaining: 3.48s
100:	learn: 0.1546393	test: 0.1529687	best: 0.1529687 (100)	total: 193ms	remaining: 1.72s
200:	learn: 0.1528213	test: 0.1517332	best: 0.1517332 (200)	total: 351ms	remaining: 1.4s
300:	learn: 0.1517532	test: 0.1514386	best: 0.1514000 (288)	total: 514ms	remaining: 1.19s
400:	learn: 0.1509722	test: 0.1515508	best: 0.1514000 (288)	total: 694ms	remaining: 1.04s
500:	learn: 0.1502967	test: 0.1516166	best: 0.1514000 (288)	total: 903ms	remaining: 899ms
600:	learn: 0.1499412	test: 0.1516849	best: 0.1514000 (288)	total: 1.08s	remaining: 719ms
700:	learn: 0.1494768	test: 0.1516928	best: 0.1514000 (288)	total: 1.26s	remaining: 539ms
800:	learn: 0.1490851	test: 0.1518297	best: 0.1514000 (288)	total: 1.43s	remaining: 356ms
900:	learn: 0.1487173	test: 0.1519119	best: 0.1514000 (288)	total: 1.61s	remaining: 177ms
999:	learn: 0.1483869	test: 0.1519628	best: 0.1514000 (288)	total: 1.78s	r

In [105]:
model.predict(test[X])

array([45436.28801924, 21738.83525716, 16205.32627896, ...,
       28444.11570854, 16205.32627896, 27824.78614511])

In [106]:
test['price_pred'] = model.predict(test[X]) 

In [107]:
test

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize,price_pred
4984,X6,2019,52990,Semi-Auto,3086,Diesel,145,34.9,3.0,45436.288019
4880,X1,2018,24081,Semi-Auto,13245,Diesel,150,60.1,2.0,21738.835257
9132,1 Series,2016,12999,Automatic,68949,Petrol,200,43.5,2.0,16205.326279
6451,1 Series,2020,11995,Semi-Auto,10,Petrol,150,34.5,2.0,32088.707415
7199,3 Series,2020,29875,Semi-Auto,150,Petrol,145,42.2,2.0,32088.707415
...,...,...,...,...,...,...,...,...,...,...
9627,3 Series,2015,14999,Automatic,78680,Diesel,160,52.3,3.0,19357.530409
6096,4 Series,2019,29676,Automatic,7365,Petrol,145,41.5,3.0,44808.449332
1747,5 Series,2019,30570,Automatic,3067,Hybrid,145,49.6,2.0,28444.115709
4023,3 Series,2016,14999,Automatic,70054,Hybrid,0,148.7,2.0,16205.326279


## Checking model

In [108]:
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

In [109]:
def error(y_true,y_pred):
  print(mean_absolute_error(y_true,y_pred))
  print(mean_absolute_percentage_error(y_true,y_pred))

In [110]:
error(test['price'], test['price_pred'])

3366.679482256595
0.15363582890850797


# Learn with all features

In [111]:
X = ['model', 'year', 'transmission', 'mileage', 'fuelType', 'tax',
       'mpg', 'engineSize']

cat_features = ['model', 'transmission', 'fuelType']

y = ['price']

In [112]:
parameters = {'cat_features': cat_features,
              'eval_metric': 'MAPE',
              'random_seed':42,
              'verbose':100}

In [113]:
model = CatBoostRegressor(**parameters)

In [114]:
model.fit(train[X],train[y],eval_set=(val[X],val[y]))

Learning rate set to 0.068263
0:	learn: 0.4607438	test: 0.4563750	best: 0.4563750 (0)	total: 4.47ms	remaining: 4.47s
100:	learn: 0.0904302	test: 0.0896388	best: 0.0896388 (100)	total: 300ms	remaining: 2.67s
200:	learn: 0.0781784	test: 0.0803125	best: 0.0803125 (200)	total: 557ms	remaining: 2.21s
300:	learn: 0.0729925	test: 0.0764506	best: 0.0764506 (300)	total: 826ms	remaining: 1.92s
400:	learn: 0.0695552	test: 0.0744676	best: 0.0744553 (399)	total: 1.11s	remaining: 1.66s
500:	learn: 0.0670683	test: 0.0732827	best: 0.0732827 (500)	total: 1.41s	remaining: 1.4s
600:	learn: 0.0652835	test: 0.0726170	best: 0.0726128 (597)	total: 1.71s	remaining: 1.13s
700:	learn: 0.0636320	test: 0.0720622	best: 0.0720500 (695)	total: 2.06s	remaining: 877ms
800:	learn: 0.0623408	test: 0.0714586	best: 0.0714329 (794)	total: 2.41s	remaining: 598ms
900:	learn: 0.0611487	test: 0.0711564	best: 0.0711321 (898)	total: 2.72s	remaining: 298ms
999:	learn: 0.0601407	test: 0.0708439	best: 0.0708403 (998)	total: 3.08s	r

## Checking model

In [115]:
test['price_pred_all'] = model.predict(test[X])

In [116]:
error(test['price'],test['price_pred_all'])

1556.5378327680894
0.07367840910148211


# Number of iterations and learning rate

## Reducing the learning rate

In [124]:
X = ['model', 'year', 'transmission', 'mileage', 'fuelType', 'tax',
       'mpg', 'engineSize']

cat_features = ['model', 'transmission', 'fuelType']

y = ['price']

In [125]:
model = CatBoostRegressor(cat_features=cat_features,
                          learning_rate = 0.03,
                          eval_metric='MAPE',
                          random_seed=42,
                          verbose=100)

In [126]:
model.fit(train[X],train[y],eval_set=(val[X],val[y]))

0:	learn: 0.4748747	test: 0.4703283	best: 0.4703283 (0)	total: 6.04ms	remaining: 6.04s
100:	learn: 0.1209943	test: 0.1173861	best: 0.1173861 (100)	total: 319ms	remaining: 2.84s
200:	learn: 0.0917193	test: 0.0895708	best: 0.0895708 (200)	total: 601ms	remaining: 2.39s
300:	learn: 0.0831827	test: 0.0819571	best: 0.0819571 (300)	total: 883ms	remaining: 2.05s
400:	learn: 0.0787966	test: 0.0787471	best: 0.0787471 (400)	total: 1.21s	remaining: 1.81s
500:	learn: 0.0756785	test: 0.0766496	best: 0.0766496 (500)	total: 1.55s	remaining: 1.54s
600:	learn: 0.0734238	test: 0.0751882	best: 0.0751882 (600)	total: 1.86s	remaining: 1.23s
700:	learn: 0.0716668	test: 0.0740858	best: 0.0740858 (700)	total: 2.13s	remaining: 911ms
800:	learn: 0.0702486	test: 0.0731632	best: 0.0731632 (800)	total: 2.44s	remaining: 607ms
900:	learn: 0.0687932	test: 0.0723839	best: 0.0723839 (900)	total: 2.8s	remaining: 307ms
999:	learn: 0.0676111	test: 0.0718199	best: 0.0718198 (997)	total: 3.14s	remaining: 0us

bestTest = 0.07

## Increasing the learning rate

In [127]:
X = ['model', 'year', 'transmission', 'mileage', 'fuelType', 'tax',
       'mpg', 'engineSize']

cat_features = ['model', 'transmission', 'fuelType']

y = ['price']

In [129]:
parameters = {'cat_features': cat_features,
              'eval_metric': 'MAPE',
              'learning_rate': 0.08,
              'random_seed':42,
              'verbose':100}

In [130]:
model = CatBoostRegressor(**parameters)

In [131]:
model.fit(train[X],train[y],
          eval_set=(val[X],val[y]))

0:	learn: 0.4564416	test: 0.4521294	best: 0.4521294 (0)	total: 7.03ms	remaining: 7.03s
100:	learn: 0.0875290	test: 0.0867329	best: 0.0867329 (100)	total: 353ms	remaining: 3.15s
200:	learn: 0.0760834	test: 0.0779235	best: 0.0779235 (200)	total: 645ms	remaining: 2.56s
300:	learn: 0.0708670	test: 0.0740290	best: 0.0740290 (300)	total: 929ms	remaining: 2.16s
400:	learn: 0.0676240	test: 0.0718814	best: 0.0718814 (400)	total: 1.24s	remaining: 1.85s
500:	learn: 0.0654070	test: 0.0709221	best: 0.0708594 (496)	total: 1.52s	remaining: 1.51s
600:	learn: 0.0638577	test: 0.0704606	best: 0.0704490 (599)	total: 1.81s	remaining: 1.2s
700:	learn: 0.0622969	test: 0.0698226	best: 0.0698226 (700)	total: 2.1s	remaining: 896ms
800:	learn: 0.0609583	test: 0.0694169	best: 0.0694004 (791)	total: 2.42s	remaining: 600ms
900:	learn: 0.0596089	test: 0.0689968	best: 0.0689945 (895)	total: 2.73s	remaining: 300ms
999:	learn: 0.0584626	test: 0.0686829	best: 0.0686377 (993)	total: 3.11s	remaining: 0us

bestTest = 0.068

# Learn on all data

In [132]:
len(train)

6468

In [133]:
len(val)

2156

In [134]:
len(train) + len(val)

8624

In [135]:
train_full = pd.concat([train,val])

In [136]:
X = ['model', 'year', 'transmission', 'mileage', 'fuelType', 'tax',
       'mpg', 'engineSize']

cat_features = ['model', 'transmission', 'fuelType']

y = ['price']

In [137]:
parameters = {
    'cat_features': cat_features,
    'eval_metric': 'MAPE',
    'learning_rate': 0.08,
    'random_seed':42,
    'verbose':100
}

In [138]:
model = CatBoostRegressor(**parameters)

In [139]:
model.fit(train[X],train[y],eval_set=(val[X],val[y]))

0:	learn: 0.4564416	test: 0.4521294	best: 0.4521294 (0)	total: 4.31ms	remaining: 4.3s
100:	learn: 0.0875290	test: 0.0867329	best: 0.0867329 (100)	total: 339ms	remaining: 3.02s
200:	learn: 0.0760834	test: 0.0779235	best: 0.0779235 (200)	total: 656ms	remaining: 2.61s
300:	learn: 0.0708670	test: 0.0740290	best: 0.0740290 (300)	total: 998ms	remaining: 2.32s
400:	learn: 0.0676240	test: 0.0718814	best: 0.0718814 (400)	total: 1.35s	remaining: 2.02s
500:	learn: 0.0654070	test: 0.0709221	best: 0.0708594 (496)	total: 1.66s	remaining: 1.65s
600:	learn: 0.0638577	test: 0.0704606	best: 0.0704490 (599)	total: 1.93s	remaining: 1.28s
700:	learn: 0.0622969	test: 0.0698226	best: 0.0698226 (700)	total: 2.22s	remaining: 947ms
800:	learn: 0.0609583	test: 0.0694169	best: 0.0694004 (791)	total: 2.52s	remaining: 627ms
900:	learn: 0.0596089	test: 0.0689968	best: 0.0689945 (895)	total: 2.83s	remaining: 311ms
999:	learn: 0.0584626	test: 0.0686829	best: 0.0686377 (993)	total: 3.12s	remaining: 0us

bestTest = 0.06

In [143]:
model.best_iteration_

In [144]:
parameters

{'cat_features': ['model', 'transmission', 'fuelType'],
 'eval_metric': 'MAPE',
 'learning_rate': 0.08,
 'random_seed': 42,
 'verbose': 100}

In [145]:
model = CatBoostRegressor(**parameters)

In [146]:
model.fit(train_full[X],train_full[y])

0:	learn: 0.4538016	total: 5.16ms	remaining: 5.16s
100:	learn: 0.0854178	total: 322ms	remaining: 2.86s
200:	learn: 0.0745022	total: 630ms	remaining: 2.5s
300:	learn: 0.0701717	total: 939ms	remaining: 2.18s
400:	learn: 0.0676577	total: 1.24s	remaining: 1.85s
500:	learn: 0.0656926	total: 1.53s	remaining: 1.52s
600:	learn: 0.0638104	total: 1.87s	remaining: 1.24s
700:	learn: 0.0624302	total: 2.22s	remaining: 947ms
800:	learn: 0.0612275	total: 2.57s	remaining: 640ms
900:	learn: 0.0599823	total: 2.9s	remaining: 319ms
999:	learn: 0.0589827	total: 3.31s	remaining: 0us


In [147]:
test['price_pred_all_features_and_data'] = model.predict(test[X])

In [148]:
error(test['price'],test['price_pred_all'])

1556.5378327680894
0.07367840910148211


# Changing the optimization function

In [149]:
X = ['model', 'year', 'transmission', 'mileage', 'fuelType', 'tax',
       'mpg', 'engineSize']

cat_features = ['model', 'transmission', 'fuelType']

y = ['price']

In [150]:
parameters = {'cat_features': cat_features,
              'loss_function': 'MAE',
              'eval_metric': 'MAPE',
              'learning_rate': 0.1,
              'random_seed':42,
              'verbose':100}

In [151]:
model = CatBoostRegressor(**parameters)

In [152]:
model.fit(train[X],train[y],eval_set=(val[X],val[y]))

0:	learn: 0.3849345	test: 0.3793341	best: 0.3793341 (0)	total: 5.2ms	remaining: 5.19s
100:	learn: 0.0764009	test: 0.0774578	best: 0.0774578 (100)	total: 384ms	remaining: 3.42s
200:	learn: 0.0666216	test: 0.0706498	best: 0.0706498 (200)	total: 732ms	remaining: 2.91s
300:	learn: 0.0628577	test: 0.0695086	best: 0.0694924 (298)	total: 1.08s	remaining: 2.51s
400:	learn: 0.0605748	test: 0.0687890	best: 0.0687826 (399)	total: 1.48s	remaining: 2.21s
500:	learn: 0.0589764	test: 0.0683467	best: 0.0683222 (490)	total: 1.81s	remaining: 1.8s
600:	learn: 0.0579201	test: 0.0680567	best: 0.0680567 (600)	total: 2.15s	remaining: 1.43s
700:	learn: 0.0570454	test: 0.0678671	best: 0.0678659 (681)	total: 2.53s	remaining: 1.08s
800:	learn: 0.0563072	test: 0.0676018	best: 0.0676018 (800)	total: 2.86s	remaining: 710ms
900:	learn: 0.0555798	test: 0.0674500	best: 0.0674384 (889)	total: 3.18s	remaining: 350ms
999:	learn: 0.0548863	test: 0.0673326	best: 0.0672972 (972)	total: 3.5s	remaining: 0us

bestTest = 0.0672